In [1]:
from math import log2
from util import flatten
from felt import Felt
from hypercube import Hypercube
from mle import MultiLinearExtension
from sumcheck import SumcheckProtocol, SuperEfficientProver, SuperEfficientVerifier

In [2]:
a = [[Felt(0), Felt(1)],
    [Felt(2), Felt(0)]]
b = [[Felt(1), Felt(0)],
    [Felt(0), Felt(4)]]
c = [[Felt(0), Felt(4)],
    [Felt(2), Felt(0)]]
n = 2

mlea = MultiLinearExtension(flatten(a))
mleb = MultiLinearExtension(flatten(b))
mlec = MultiLinearExtension(flatten(c))


g = MultiLinearExtension([
    mlea.eval([hc[0],Felt(0)]) * mleb.eval([Felt(0),hc[1]]) +
    mlea.eval([hc[0],Felt(1)]) * mleb.eval([Felt(1),hc[1]])
    for hc in Hypercube(n)
])

fc = MultiLinearExtension([
    mlec.eval([hc[0],hc[1]])
    for hc in Hypercube(2)
])

c = fc.hypercube_eval()
sc = SumcheckProtocol(
    verifier=SuperEfficientVerifier(2,lambda x: g.eval(x)),
    prover=SuperEfficientProver(g.evals,c,2),
    variables=int(log2(len(g.evals))),
)
sc.execute()

Round 1
Round 2 Felt(212892565379185179,2305843009213693951)
Verfied!


[Felt(212892565379185179,2305843009213693951),
 Felt(513962219604624877,2305843009213693951)]

In [3]:
n = 4

a = [[Felt.random() for _ in range(n)] for _ in range(n)]
b = [[Felt.random() for _ in range(n)] for _ in range(n)]
c = [[Felt(0) for _ in range(n)] for _ in range(n)]

for i in range(n):
    for j in range(n):
        for k in range(n):
            c[i][j] += a[i][k] * b[k][j]

mlea = MultiLinearExtension(flatten(a))
mleb = MultiLinearExtension(flatten(b))
mlec = MultiLinearExtension(flatten(c))

ln = int(log2(len(mlec.evals)))

def oracle(rs):
    return sum([
        mlea.eval(rs[:len(rs)//2]+hc) * 
        mleb.eval(hc+rs[len(rs)//2:]) 
        for hc in Hypercube(len(rs)//2)
    ],Felt(0))

c1 = sum(mlec.evals, Felt(0))
sc = SumcheckProtocol(
    verifier=SuperEfficientVerifier(2,oracle),
    prover=SuperEfficientProver(mlec.evals,c1,2),
    variables=ln,
)
sc.execute()

Round 1
Round 2 Felt(1808044768669136197,2305843009213693951)
Round 3 Felt(1396261383884423451,2305843009213693951)
Round 4 Felt(1663103431478705399,2305843009213693951)
Verfied!


[Felt(1808044768669136197,2305843009213693951),
 Felt(1396261383884423451,2305843009213693951),
 Felt(1663103431478705399,2305843009213693951),
 Felt(427315058694695055,2305843009213693951)]